# 신발 추천 서비스 : JordanRecommend

In [ ]:
import pandas as pd
import pickle

# 피클에서 df 불러오기
with open("jordan_embedding_dict_df.pkl","rb") as f:
    df = pickle.load(f)
df

In [ ]:
class JordanRecommend:
    
    
    
    def __init__(self,k_num=8,opt=False, random_state=42):
        
#         print('__init__ 실행')

        import pandas as pd
        import pickle
        from sklearn.cluster import KMeans
        

        # 피클에서 df 불러오기
        with open("jordan_embedding_dict_df.pkl","rb") as f:
            self.__df = pickle.load(f)
            
        # 변하지 않는 멤버 변수 초기화 
        self.df = self.__df.loc[:,0:511] # 임베딩 벡터 
        self.df_ = self.__df.loc[:,['model_code','file_path']] # 모델 이름, 이미지파일 위치
        
        # 변하는 멤버변수 초기화
        self.dfm = self.df.copy()
        self.df_m = self.df_.copy()
        
        # k_num, opt, random_state 초기화
        self.opt = opt
        self.random_state = random_state

#         if self.opt == True:
#             self.silhouette()
#         else:
#             self.k_num = k_num
        self.k_num = k_num
    
    
        
        # kmean 초기화
        self.kmeans = KMeans(n_clusters=self.k_num, random_state=self.random_state,verbose=0)
        self.labels = self.kmeans.fit_predict(self.dfm)
        self.df_m['cluster'] = self.labels # df_m 의 컬럼 ['index', 'model_code','file_path', 'cluster']
        
        
        self.show_samples(self.dfm, self.df_m)
        
        
        
    def show_samples(self,dfm,df_m):
#         print('show_samples 실행')
        import matplotlib.pyplot as plt
        from PIL import Image
        import random
        
        print('현재 k_num : '+str(self.k_num))
        print('현재 df_m 개수 : '+str(len(self.df_m)))
        
        self.dfm = dfm
        self.df_m = df_m
        
        # 현재 k_num 을 바탕으로 현재 df_m 에 존재하는 클러스터 별 샘플 그리기
        
        if self.k_num < len(self.df_m):

            fig, ax = plt.subplots(1,self.k_num)
            fig.set_size_inches(20,2)
            
            for i in range(self.k_num): # 여기 위치에서는 k_num 과 cluster 수가 같아야한다.

                df = self.return_cluster_num_df(i+1) # for 문 i 에 따라 df가 변경

                img_loc = df.file_path.tolist()
                random.shuffle(img_loc)
                ax[i].axis('off')
                ax[i].set_title(f'{i+1}')
                img = Image.open(img_loc[0])
                ax[i].imshow(img)

            plt.show()
            
            self.recommend()
        
        
        
        else: # self.k_num >= len(self.df_m) 일 때
            print('************************here is my recommend************************')

            fig, ax = plt.subplots(1,len(self.df_m))
            fig.set_size_inches(20,2)
            
            if len(self.df_m) ==1: # ax가 1개 이므로 ax[1] 라 표현하면 not subscriptable error 발생
                for j,k in zip(self.df_m['file_path'], self.df_m['model_code']):

                    ax.axis('off')
                    ax.set_title(k)
                    img = Image.open(j)
                    ax.imshow(img)

                plt.show()

            else:
                
                for i,j,k in zip(range(len(self.df_m)), self.df_m['file_path'], self.df_m['model_code']):

                    ax[i].axis('off')
                    ax[i].set_title(k)
                    img = Image.open(j)
                    ax[i].imshow(img)

                plt.show()
                
            print('\n********************************************************************\n')

            return 0


            
            
            
    def recommend(self):
#         print('recommend 실행')
        self.input1 = input('Enter the "number" of your most preferred sample or Type "re" to resuffle & show => input : ')
        print('현재 k_num : '+str(self.k_num))
        try:

            if int(self.input1) <= self.k_num:
                self.select(int(self.input1))

            else:
                print("\n\n*****************************************************************************")
                print("************************Please enter a number under "+str(self.k_num)+"************************")
                print("*****************************************************************************\n\n")

                self.recommend()


        except:
            
            if self.input1 == 're':
                print('\n\n***********************************************************')
                print('************************reshuffling************************')
                print('***********************************************************\n\n')
                self.show_samples(self.dfm,self.df_m)
                self.recommend()

            else:
                print('\n\n************************************************************************')
                print('************************Please enter it correctly************************')
                print('************************************************************************\n\n')
                self.recommend()



    def select(self,num):
        
#         print('select 실행')
        import matplotlib.pyplot as plt
        from PIL import Image
        import random
        from sklearn.cluster import KMeans



        self.silhouette() # opt = True일 때 self.k_num 을 최적으로 결정

        if self.k_num > len(self.df_m):
            self.show_samples(self.dfm, self.df_m)
        else:
            
            self.df_m = self.return_cluster_num_df(num)
            self.dfm = self.df.loc[self.df_m.index]
            
            
            if self.k_num > len(self.df_m):
                self.show_samples(self.dfm, self.df_m)

            else:
                self.kmeans = KMeans(n_clusters=self.k_num, random_state=self.random_state,verbose=0)
                self.labels = self.kmeans.fit_predict(self.dfm)
                self.df_m = self.df_m.drop(columns=['cluster'])
                self.df_m['cluster'] = self.labels
                self.dfm = self.df.loc[self.df_m.index]

                self.show_samples(self.dfm, self.df_m)


        
        
    def return_cluster_num_df(self,num): # num에 해당하는 클러스터 df를 반환하는 함수 
        
        return self.df_m[self.df_m['cluster']==num-1] # -1 의미 : 클러스터가 0부터 표현되기 때문
        

        
    def silhouette(self):

        import copy
        import numpy as np
        
        
        if self.opt==True:
        
            from sklearn.cluster import KMeans
            from sklearn.metrics import silhouette_samples, silhouette_score
            import pandas as pd
            import copy
            average_score_dict = dict()

            try:
                print('최적의 분류를 위해 silhouette 계수를 계산중입니다. 잠시만 기다려주세요')
                for i in range(4,10):

                    df0 = copy.deepcopy(self.dfm)
                    kmeans = KMeans(n_clusters= i,random_state=self.random_state,verbose=0)
                    labels = kmeans.fit_predict(df0.loc[:,0:511])
                    df0['cluster'] = labels

                    # 모든 데이터의 평균 실루엣 계수값을 구함. 
                    average_score = silhouette_score(df0.loc[:,0:511], labels)
                    average_score_dict[f'{i}'] = average_score

                self.k_num = np.array(list(average_score_dict.values())).argmax()+4
                print('실루엣 후 k_num = ' +str(self.k_num))
            except:
                pass

        else:
            pass
        

In [ ]:
# 서비스 시작을 위해 아래의 코드를 실행해주세요

In [ ]:
JordanRecommend(k_num=6,opt=True,random_state=50) # opt=True 일 때 k_num 은 무시됨, k_num=8 random_state=42 opt =False 이 default